In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = ''
resume = './log/style2/128/b0/siamese2/checkpoint.pth.tar'

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 300
start_epoch = 0
train_batch = 64
test_batch = 64
lr = 0.04
schedule = [75,150,225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/siamese2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# constrastive
thresh = 0.5

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_ = SiameseNetworkDataset(datasets.ImageFolder(val_dir), transform=val_aug, should_invert=False)
val_loader = DataLoader(val_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [10]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    model.load_state_dict(pt)

In [11]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.17M


# Loss

In [12]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


In [14]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

        # cutmix
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            target_index = targets[targets==0]
            target_index = target_index.long().cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs0.size(), lam)
            inputs0[target_index, :, bbx1:bbx2, bby1:bby2] = inputs1[target_index, :, bbx1:bbx2, bby1:bby2]
        
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        
        loss = criterion(outputs0, outputs1, targets)
            
        # compute output
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            loss = criterion(outputs0, outputs1, targets)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            # measure accuracy and record loss
            pred = outputs.data
            pred[pred < thresh] = 0.
            pred[pred >= thresh] = 1.
            prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                        batch=batch_idx + 1,
                        size=len(val_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=bar.elapsed_td,
                        eta=bar.eta_td,
                        loss=losses.avg,
                        top1=top1.avg,)
            bar.next()
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
             batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
        bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [18]:
for epoch in range(start_epoch, epochs):    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)

    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [161 | 300] LR: 0.000011
1/1211 Data:2.572 | Batch:5.953 | Total:0:00:05 | ETA:2:00:04 | Loss:1.671506106504239e-05 | top1:1.0
2/1211 Data:0.015 | Batch:0.810 | Total:0:00:06 | ETA:1:08:09 | Loss:3.818516961473506e-05 | top1:1.0
3/1211 Data:0.015 | Batch:0.613 | Total:0:00:07 | ETA:0:49:31 | Loss:3.1801979275769554e-05 | top1:1.0
4/1211 Data:0.021 | Batch:0.566 | Total:0:00:07 | ETA:0:39:57 | Loss:2.8353902507660678e-05 | top1:1.0
5/1211 Data:0.020 | Batch:0.617 | Total:0:00:08 | ETA:0:34:25 | Loss:2.5076253950828687e-05 | top1:1.0
6/1211 Data:0.018 | Batch:0.685 | Total:0:00:09 | ETA:0:30:57 | Loss:2.2608662220591214e-05 | top1:1.0
7/1211 Data:0.013 | Batch:0.603 | Total:0:00:09 | ETA:0:28:14 | Loss:2.2085404582737412e-05 | top1:1.0
8/1211 Data:0.032 | Batch:0.626 | Total:0:00:10 | ETA:0:26:16 | Loss:0.0002265672412704589 | top1:1.0
9/1211 Data:0.026 | Batch:0.466 | Total:0:00:10 | ETA:0:24:22 | Loss:0.00020245702914204836 | top1:1.0
10/1211 Data:0.040 | Batch:0.568 | Total:0:

73/1211 Data:0.025 | Batch:0.406 | Total:0:00:45 | ETA:0:09:05 | Loss:0.0012522387016410716 | top1:0.9987157534246576
74/1211 Data:0.011 | Batch:0.592 | Total:0:00:46 | ETA:0:08:27 | Loss:0.0012357658263656933 | top1:0.9987331081081081
75/1211 Data:0.024 | Batch:0.563 | Total:0:00:46 | ETA:0:08:27 | Loss:0.0012195714398937222 | top1:0.99875
76/1211 Data:0.011 | Batch:0.615 | Total:0:00:47 | ETA:0:08:57 | Loss:0.0012037889734624752 | top1:0.998766447368421
77/1211 Data:0.020 | Batch:0.492 | Total:0:00:47 | ETA:0:08:56 | Loss:0.0011883522430052127 | top1:0.9987824675324676
78/1211 Data:0.011 | Batch:0.415 | Total:0:00:48 | ETA:0:08:56 | Loss:0.0011733533536780437 | top1:0.9987980769230769
79/1211 Data:0.019 | Batch:0.620 | Total:0:00:48 | ETA:0:09:15 | Loss:0.001159030127463309 | top1:0.9988132911392406
80/1211 Data:0.008 | Batch:0.381 | Total:0:00:49 | ETA:0:09:15 | Loss:0.0011449984482737817 | top1:0.998828125
81/1211 Data:0.010 | Batch:0.393 | Total:0:00:49 | ETA:0:09:14 | Loss:0.0011

143/1211 Data:0.020 | Batch:0.514 | Total:0:01:18 | ETA:0:09:50 | Loss:0.001210573931810079 | top1:0.9987980769230769
144/1211 Data:0.006 | Batch:0.563 | Total:0:01:18 | ETA:0:09:42 | Loss:0.001202337237108016 | top1:0.9988064236111112
145/1211 Data:0.010 | Batch:0.535 | Total:0:01:19 | ETA:0:09:42 | Loss:0.0011942034160867855 | top1:0.9988146551724137
146/1211 Data:0.007 | Batch:0.347 | Total:0:01:19 | ETA:0:09:41 | Loss:0.0011861783066905607 | top1:0.9988227739726028
147/1211 Data:0.011 | Batch:0.366 | Total:0:01:20 | ETA:0:09:12 | Loss:0.0011782973018927472 | top1:0.9988307823129252
148/1211 Data:0.011 | Batch:0.396 | Total:0:01:20 | ETA:0:09:11 | Loss:0.0011866725735180316 | top1:0.9988386824324325
149/1211 Data:0.011 | Batch:0.540 | Total:0:01:20 | ETA:0:09:11 | Loss:0.0011788089747561943 | top1:0.9988464765100671
150/1211 Data:0.009 | Batch:0.562 | Total:0:01:21 | ETA:0:08:50 | Loss:0.0011711522002951823 | top1:0.9988541666666667
151/1211 Data:0.009 | Batch:0.421 | Total:0:01:21 

213/1211 Data:0.011 | Batch:0.430 | Total:0:01:51 | ETA:0:08:07 | Loss:0.0013499905512681309 | top1:0.998606220657277
214/1211 Data:0.010 | Batch:0.436 | Total:0:01:52 | ETA:0:08:07 | Loss:0.0013438099774596755 | top1:0.9986127336448598
215/1211 Data:0.009 | Batch:0.405 | Total:0:01:52 | ETA:0:07:43 | Loss:0.0013376838034716012 | top1:0.9986191860465117
216/1211 Data:0.010 | Batch:0.413 | Total:0:01:52 | ETA:0:07:43 | Loss:0.0013330146822159736 | top1:0.9986255787037037
217/1211 Data:0.027 | Batch:0.518 | Total:0:01:53 | ETA:0:07:42 | Loss:0.0013270240242784512 | top1:0.9986319124423964
218/1211 Data:0.009 | Batch:0.551 | Total:0:01:54 | ETA:0:07:46 | Loss:0.0013250905547866174 | top1:0.9986381880733946
219/1211 Data:0.011 | Batch:0.469 | Total:0:01:54 | ETA:0:07:45 | Loss:0.0013191235687883495 | top1:0.998644406392694
220/1211 Data:0.009 | Batch:0.453 | Total:0:01:54 | ETA:0:07:45 | Loss:0.0013346808291812935 | top1:0.9985795454545454
221/1211 Data:0.010 | Batch:0.442 | Total:0:01:55 

283/1211 Data:0.011 | Batch:0.465 | Total:0:02:28 | ETA:0:08:23 | Loss:0.0014975491710332012 | top1:0.9983436395759717
284/1211 Data:0.016 | Batch:0.510 | Total:0:02:28 | ETA:0:08:22 | Loss:0.0014924367313840416 | top1:0.9983494718309859
285/1211 Data:0.010 | Batch:0.404 | Total:0:02:29 | ETA:0:08:21 | Loss:0.0015240286579443676 | top1:0.9983004385964912
286/1211 Data:0.009 | Batch:0.608 | Total:0:02:29 | ETA:0:08:13 | Loss:0.0015189816808170353 | top1:0.9983063811188811
287/1211 Data:0.019 | Batch:0.552 | Total:0:02:30 | ETA:0:08:12 | Loss:0.0015138143840562953 | top1:0.9983122822299652
288/1211 Data:0.011 | Batch:0.518 | Total:0:02:30 | ETA:0:08:04 | Loss:0.001508651902803927 | top1:0.9983181423611112
289/1211 Data:0.011 | Batch:0.453 | Total:0:02:31 | ETA:0:08:04 | Loss:0.0015035191609656467 | top1:0.9983239619377162
290/1211 Data:0.010 | Batch:0.747 | Total:0:02:31 | ETA:0:07:58 | Loss:0.0014984450684723612 | top1:0.9983297413793103
291/1211 Data:0.012 | Batch:0.562 | Total:0:02:32

...

540/1211 Data:0.011 | Batch:0.418 | Total:0:04:18 | ETA:0:04:31 | Loss:0.0017410668186618784 | top1:0.9980902777777778
541/1211 Data:0.008 | Batch:0.407 | Total:0:04:18 | ETA:0:04:31 | Loss:0.0017378977179360003 | top1:0.9980938077634011
542/1211 Data:0.011 | Batch:0.351 | Total:0:04:19 | ETA:0:04:31 | Loss:0.0017594170610236253 | top1:0.9980684963099631
543/1211 Data:0.007 | Batch:0.367 | Total:0:04:19 | ETA:0:04:25 | Loss:0.0017562006936978811 | top1:0.9980720534069981
544/1211 Data:0.010 | Batch:0.400 | Total:0:04:20 | ETA:0:04:24 | Loss:0.0017790148283538924 | top1:0.998046875
545/1211 Data:0.008 | Batch:0.391 | Total:0:04:20 | ETA:0:04:24 | Loss:0.0018036270034551532 | top1:0.9980217889908257
546/1211 Data:0.009 | Batch:0.436 | Total:0:04:20 | ETA:0:04:25 | Loss:0.0018293230226972077 | top1:0.9979967948717948
547/1211 Data:0.007 | Batch:0.414 | Total:0:04:21 | ETA:0:04:25 | Loss:0.001826012500676327 | top1:0.9980004570383912
548/1211 Data:0.018 | Batch:0.556 | Total:0:04:21 | ETA:

610/1211 Data:0.008 | Batch:0.358 | Total:0:04:47 | ETA:0:04:07 | Loss:0.0019016287212014235 | top1:0.9978995901639345
611/1211 Data:0.011 | Batch:0.405 | Total:0:04:47 | ETA:0:04:07 | Loss:0.001898551859448476 | top1:0.9979030278232406
612/1211 Data:0.008 | Batch:0.443 | Total:0:04:48 | ETA:0:04:09 | Loss:0.0018954809223915242 | top1:0.997906454248366
613/1211 Data:0.009 | Batch:0.385 | Total:0:04:48 | ETA:0:04:09 | Loss:0.0018924719587456943 | top1:0.9979098694942904
614/1211 Data:0.010 | Batch:0.374 | Total:0:04:49 | ETA:0:04:08 | Loss:0.0018895750793857505 | top1:0.9979132736156352
615/1211 Data:0.007 | Batch:0.390 | Total:0:04:49 | ETA:0:03:57 | Loss:0.001886519614086362 | top1:0.9979166666666667
616/1211 Data:0.008 | Batch:0.389 | Total:0:04:49 | ETA:0:03:57 | Loss:0.001883488605987221 | top1:0.9979200487012987
617/1211 Data:0.011 | Batch:0.377 | Total:0:04:50 | ETA:0:03:57 | Loss:0.0018805616087359295 | top1:0.9979234197730956
618/1211 Data:0.011 | Batch:0.383 | Total:0:04:50 | 

680/1211 Data:0.021 | Batch:0.396 | Total:0:05:16 | ETA:0:03:34 | Loss:0.0019199042954436236 | top1:0.9978860294117647
681/1211 Data:0.006 | Batch:0.388 | Total:0:05:16 | ETA:0:03:33 | Loss:0.0019171413393455083 | top1:0.9978891336270191
682/1211 Data:0.009 | Batch:0.434 | Total:0:05:16 | ETA:0:03:32 | Loss:0.0019170394893030136 | top1:0.997892228739003
683/1211 Data:0.008 | Batch:0.469 | Total:0:05:17 | ETA:0:03:32 | Loss:0.0019142603415071779 | top1:0.9978953147877013
684/1211 Data:0.009 | Batch:0.440 | Total:0:05:17 | ETA:0:03:41 | Loss:0.001911651136855366 | top1:0.9978983918128655
685/1211 Data:0.011 | Batch:0.447 | Total:0:05:18 | ETA:0:03:40 | Loss:0.0019089116019761997 | top1:0.9979014598540146
686/1211 Data:0.010 | Batch:0.494 | Total:0:05:18 | ETA:0:03:40 | Loss:0.001906169336116337 | top1:0.9979045189504373
687/1211 Data:0.006 | Batch:0.449 | Total:0:05:19 | ETA:0:03:46 | Loss:0.0019034532878755255 | top1:0.9979075691411936
688/1211 Data:0.009 | Batch:0.416 | Total:0:05:19 |

750/1211 Data:0.024 | Batch:0.476 | Total:0:05:45 | ETA:0:03:19 | Loss:0.0019000491408720942 | top1:0.9979166666666667
751/1211 Data:0.008 | Batch:0.388 | Total:0:05:46 | ETA:0:03:18 | Loss:0.0019181215469151588 | top1:0.9978986351531292
752/1211 Data:0.007 | Batch:0.395 | Total:0:05:46 | ETA:0:03:18 | Loss:0.0019156575617877082 | top1:0.9979014295212766
753/1211 Data:0.008 | Batch:0.360 | Total:0:05:47 | ETA:0:03:03 | Loss:0.0019334960121657734 | top1:0.9978834661354582
754/1211 Data:0.008 | Batch:0.389 | Total:0:05:47 | ETA:0:03:03 | Loss:0.0019309514486270378 | top1:0.997886273209549
755/1211 Data:0.010 | Batch:0.518 | Total:0:05:48 | ETA:0:03:02 | Loss:0.0019284304748203036 | top1:0.9978890728476821
756/1211 Data:0.010 | Batch:0.410 | Total:0:05:48 | ETA:0:03:09 | Loss:0.0019473604112561819 | top1:0.9978711970899471
757/1211 Data:0.009 | Batch:0.441 | Total:0:05:48 | ETA:0:03:09 | Loss:0.001944818946080652 | top1:0.9978740092470277
758/1211 Data:0.010 | Batch:0.370 | Total:0:05:49 

KeyboardInterrupt: 